# **Descarga de datos mediante la API de [The Materials Project](https://next-gen.materialsproject.org/)**

## *Objetivo:* Itera sobre el archivo *ids.csv* generado por código anterior. Durante este proceso, se realizan los requests a la API de Materials Project para obtener toda la información necesaria por id y, posteriormente, se almacenan todos los datos en un DataFrame.

### 1. Se cargan librerías y se declaran variables

In [ ]:
### Se cargan librerias ###
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from mp_api.client import MPRester
key="ebMdddl0cG3BZj3VyN9qMb6Ilj4Dl6dz"
mpr = MPRester(key)

### 2. Se crea el Dataframe que fungirá como base de datos y se declaran las listas pertinentes

In [ ]:
### Se crea DataFrame ###
df=pd.DataFrame()

### Se crean listas ###
ids=[] #Lista para el ids
formule=[] #Lista para la formula quimica
NELE=[] #Lista para el numero total de elementos del compuesto
ELE=[] #Lista para los elementos del compuesto
BG=[] #Lista para el BandGap
IBGD=[] #Lista para el tipo de BandGap
IM=[] #Lista para saber si es metal
SC=[] #Lista para el tipo de estructura cristalina
a=[] #Lista para el parametro de red "a"
b=[] #Lista para el parametro de red "b"
c=[] #Lista para el parametro de red "c"
alfa=[] #Lista para el parametro de red "alpha"
beta=[] #Lista para el parametro de red "beta"
gamma=[] #Lista para el parametro de red "gamma"
CA=[] #Lista para el coeficiente de absorcion optica

### 3. Se realizan los requests mediante la API para descargar toda la información necesaria

In [ ]:
### Se realiza requests ###
with open('datos/ids.csv','r') as file:
    for material in file:
        material=str(material.strip())
        ids.append(material)
        
        docs = mpr.summary.search(material_ids=[material])
        IM.append(docs[0].is_metal)
        formule.append(docs[0].formula_pretty)
        SC.append(str(list(docs[0].symmetry)[0][-1]))
        IBGD.append(docs[0].is_gap_direct)
        NELE.append(docs[0].nelements)
        ELE.append(docs[0].chemsys)
        BG.append(docs[0].band_gap)
        a.append(docs[0].structure.lattice.abc[0])
        b.append(docs[0].structure.lattice.abc[1])
        c.append(docs[0].structure.lattice.abc[2])
        alfa.append(docs[0].structure.lattice.angles[0])
        beta.append(docs[0].structure.lattice.angles[1])
        gamma.append(docs[0].structure.lattice.angles[2])
        
        absorption = mpr.absorption.search(material_ids=[material], fields=["energies", "absorption_coefficient"])
        if len(absorption)==0:
            CA.append(np.nan)
        else:
            absorption=absorption
            energia=absorption[0].energies
            absorcion=absorption[0].absorption_coefficient
            x=np.array(absorption[0].energies)
            y=np.array(absorption[0].absorption_coefficient)
            CA.append(np.interp(2.479647809,x,y))

### 4. Se unifican las listas para generar la base de datos

In [ ]:
### Se estructura columanàs del DataFrame ###
df['id']=ids 
df['Formule']=formule
df['No Ele']=NELE
df['Ele']=ELE
df['BG']=BG
df['IM']=IM
df['Is BG Dir']=IBGD
df['Structure']=SC
df['a']=a
df['b']=b
df['c']=c
df['alfa']=alfa
df['beta']=beta
df['gamma']=gamma
df['CA']=CA

### Se exporta el DataFrame ###
#df.to_csv('datos/datos_crudos.csv',index=False) #Descomentar línea si es necesario